In [1]:
# # Install required libraries
# import os

# try:
#     import selenium
# except ImportError:
#     os.system("pip install selenium")

# try:
#     import webdriver_manager
# except ImportError:
#     os.system("pip install webdriver-manager")

# try:
#     import pandas as pd
# except ImportError:
#     os.system("pip install pandas")

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

In [2]:
# Initialize Chrome with options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")  # Start maximized for better visibility
chrome_options.add_argument("--disable-infobars")  # Disable infobars
chrome_options.add_argument("--disable-extensions")  # Disable extensions for simplicity
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

def scroll_to_bottom(driver, pause_time=2):
    """
    Scroll to the bottom of the page to load dynamic content.
    """
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(pause_time)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

driver.get("https://www.expedia.com/Hotel-Search?adults=2&d1=2025-05-01&d2=2025-05-07&destination=Seoul%20%28and%20vicinity%29%2C%20South%20Korea&endDate=2025-05-07&flexibility=0_DAY&latLong=37.565994%2C126.982577&regionId=178308&rooms=1&semdtl=&sort=RECOMMENDED&startDate=2025-05-01&stay_options_group=hotels_option&theme=&useRewards=false&userIntent=")
time.sleep(6)  # Wait for the page to load completely

# Extract hotel data
listing_container = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-stid="property-listing-results"]'))
)
hotel_links = listing_container.find_elements(By.CSS_SELECTOR, 'a[data-stid="open-hotel-information"]')
print(f"Found {len(hotel_links)} hotel links.")


Found 300 hotel links.


In [3]:
import pandas as pd

# Initialize a list to store all hotel data
hotel_data = []
location = "Seoul, South Korea"

for index, link in enumerate(hotel_links):
    try:
        href = link.get_attribute('href')
        print(f"\nProcessing link {index + 1}")

        # Open the link in a new tab using JavaScript
        driver.execute_script("window.open(arguments[0]);", href)
        time.sleep(3)  # Wait for the new tab to open

        # Switch to the new tab
        driver.switch_to.window(driver.window_handles[-1])
        time.sleep(15)  # Wait for the new page to load

        hotel_details = {"Location": location}

        # Extract Hotel Name
        try:
            # First, try to locate the hotel name with the old structure
            try:
                hotel_name_element = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "h1.uitk-heading.uitk-heading-3"))
                )
                hotel_details['Hotel Name'] = hotel_name_element.text.strip()
            except Exception:
                # If the old structure fails, try the new structure
                hotel_name_element = WebDriverWait(driver, 20).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "h1.uitk-heading.uitk-heading-4"))
                )
                hotel_details['Hotel Name'] = hotel_name_element.text.strip()

            print(f"Extracted Hotel Name: {hotel_details['Hotel Name']}")
        except Exception as e:
            print(f"Error extracting hotel name: {e}")
            hotel_details['Hotel Name'] = None

        # Extract Stars
        try:
            stars_container = driver.find_element(By.CSS_SELECTOR, "div.uitk-rating.uitk-spacing.uitk-spacing-margin-blockstart-one")
            stars_count = len(stars_container.find_elements(By.TAG_NAME, "svg"))
            hotel_details['Stars'] = stars_count
        except Exception as e:
            print(f"Error extracting stars for hotel {index + 1}: {e}")
            hotel_details['Stars'] = None

        # Extract Rating
        try:
            rating_element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, "span.uitk-badge-base-large.uitk-badge-positive > span.uitk-badge-base-text")
                )
            )
            hotel_details['Rating'] = rating_element.text.strip()
        except Exception as e:
            print(f"Error extracting rating: {e}")
            hotel_details['Rating'] = None

        # Extract Review Keyword
        try:
            review_summary_container = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-stid='content-hotel-reviewsummary']"))
            )
            review_keyword_element = review_summary_container.find_element(
                By.CSS_SELECTOR, "div.uitk-text.uitk-type-500.uitk-type-medium.uitk-text-default-theme"
            )
            review_keyword = review_keyword_element.text.strip()
            hotel_details['Review Keyword'] = review_keyword
            print(f"Extracted Review Keyword: {review_keyword}")
        except Exception as e:
            print(f"Error extracting review keyword: {e}")
            hotel_details['Review Keyword'] = None

        # Extract Services
        try:
            services_list = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "ul.uitk-typelist[data-stid='sp-content-list']"))
            )

            services_lis = services_list.find_elements(By.CSS_SELECTOR, "li[data-stid^='sp-content-item-']")
            service_count = len(services_lis)
            print(f"Number of services: {service_count}")

            services = []
            for i in range(service_count):
                try:
                    service_span = driver.find_element(
                        By.CSS_SELECTOR,
                        f"ul.uitk-typelist[data-stid='sp-content-list'] li[data-stid='sp-content-item-{i}'] span.uitk-text.uitk-type-300.uitk-text-default-theme.uitk-layout-flex-item"
                    )
                    services.append(service_span.text.strip())
                except Exception as e:
                    print(f"Error extracting service {i}: {e}")
                    services.append(None)

            hotel_details['Services'] = services
        except Exception as e:
            print(f"Error extracting services: {e}")
            hotel_details['Services'] = []

        # Extract Property Offers
        try:
            property_offers_container = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div[data-stid="section-room-list"]'))
            )

            property_offers = property_offers_container.find_elements(By.CSS_SELECTOR, 'div[data-stid^="property-offer-"]')

            offers = []
            for offer_index, offer in enumerate(property_offers, start=1):
                offer_details = {}

                try:
                    room_name_element = offer.find_element(By.CSS_SELECTOR, "h3.uitk-heading.uitk-heading-6")
                    offer_details['Room Name'] = room_name_element.text.strip()
                except Exception as e:
                    print(f"Error extracting room name for offer {offer_index}: {e}")
                    offer_details['Room Name'] = None

                try:
                    price_summary_div = offer.find_element(By.CSS_SELECTOR, 'div[data-stid="price-summary"]')
                    room_charges_element = price_summary_div.find_element(
                        By.CSS_SELECTOR,
                        "div.uitk-text.uitk-type-500.uitk-type-medium.uitk-text-emphasis-theme"
                    )
                    offer_details['Room Charges'] = room_charges_element.text.strip()
                except Exception as e:
                    print(f"Error extracting room charges for offer {offer_index}: {e}")
                    offer_details['Room Charges'] = None

                offers.append(offer_details)

            hotel_details['Property Offers'] = offers
        except Exception as e:
            print(f"Error extracting property offers: {e}")
            hotel_details['Property Offers'] = []

        # Add hotel details to list
        hotel_data.append(hotel_details)
        print(f"Extracted Hotel Data: {hotel_details}")

        # Close the new tab
        driver.close()

        # Switch back to the main tab
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(1)

    except Exception as e:
        print(f"Error processing link {index + 1}: {e}")
        continue

# Export hotel data to a CSV file
df = pd.DataFrame(hotel_data)

# Normalize property offers if necessary
if 'Property Offers' in df.columns:
    df['Property Offers'] = df['Property Offers'].apply(lambda x: ', '.join(str(offer) for offer in x) if isinstance(x, list) else x)

# Save to CSV
output_filename = f"hotel_data_{location.replace(',', '').replace(' ', '_')}.csv"
df.to_csv(output_filename, index=False)
print(f"Data exported to {output_filename}")


Processing link 1
Extracted Hotel Name: ENA Suite Hotel Namdaemun
Extracted Review Keyword: Wonderful
Number of services: 6
Error extracting room charges for offer 10: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-stid="price-summary"]"}
  (Session info: chrome=131.0.6778.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00BA34A3+25059]
	(No symbol) [0x00B2CEA4]
	(No symbol) [0x00A0BEC3]
	(No symbol) [0x00A4FD86]
	(No symbol) [0x00A4FFCB]
	(No symbol) [0x00A45B01]
	(No symbol) [0x00A71F44]
	(No symbol) [0x00A45A24]
	(No symbol) [0x00A72194]
	(No symbol) [0x00A8B51E]
	(No symbol) [0x00A71C96]
	(No symbol) [0x00A43FAC]
	(No symbol) [0x00A44F3D]
	GetHandleVerifier [0x00E95613+3113811]
	GetHandleVerifier [0x00EAA2DA+3199002]
	GetHandleVerifier [0x00EA2AB2+3168242]
	GetHandleVerifier [0x00C43310+680016]
	(


Processing link 5
Extracted Hotel Name: Shilla Stay Guro Digital Complex Station
Extracted Review Keyword: Excellent
Number of services: 6
Extracted Hotel Data: {'Location': 'Seoul, South Korea', 'Hotel Name': 'Shilla Stay Guro Digital Complex Station', 'Stars': 4, 'Rating': '8.6', 'Review Keyword': 'Excellent', 'Services': ['Parking included', 'Bar', 'Air conditioning', 'Gym', 'Free WiFi', 'Laundry'], 'Property Offers': [{'Room Name': 'Standard Double Room', 'Room Charges': '$103'}, {'Room Name': 'Deluxe Double Room (High Floor)', 'Room Charges': '$109'}, {'Room Name': 'Standard Family Twin Room', 'Room Charges': '$115'}, {'Room Name': 'Standard Twin Room', 'Room Charges': '$103'}, {'Room Name': 'Deluxe Twin Room (High Floor)', 'Room Charges': '$109'}, {'Room Name': 'Premier Standard Double Room - Free minibar', 'Room Charges': '$116'}]}

Processing link 6
Extracted Hotel Name: LOTTE City Hotel Myeongdong
Extracted Review Keyword: Wonderful
Number of services: 6
Error extracting room


Processing link 8
Extracted Hotel Name: Shilla Stay Mapo Hongdae
Extracted Review Keyword: Very good
Number of services: 6
Extracted Hotel Data: {'Location': 'Seoul, South Korea', 'Hotel Name': 'Shilla Stay Mapo Hongdae', 'Stars': 4, 'Rating': '8.4', 'Review Keyword': 'Very good', 'Services': ['Restaurant', 'Breakfast available', 'Parking available', 'Laundry', '24/7 front desk', 'Housekeeping'], 'Property Offers': [{'Room Name': 'Standard Twin Room', 'Room Charges': '$139'}, {'Room Name': 'Standard Hollywood Double', 'Room Charges': '$151'}, {'Room Name': 'Standard Double Room', 'Room Charges': '$139'}, {'Room Name': '[THE SHILLA DUTY FREE] Standard Double Room + 2 Shilla Duty Free coupons (per booking)', 'Room Charges': '$142'}, {'Room Name': '[THE SHILLA DUTY FREE] Standard Twin Room + 2 Shilla Duty Free coupons (per booking)', 'Room Charges': '$142'}, {'Room Name': 'Deluxe Double Room, City View (High Floor)', 'Room Charges': '$145'}, {'Room Name': '[THE SHILLA DUTY FREE] Deluxe D


Processing link 12
Extracted Hotel Name: Ocloud Hotel Gangnam
Extracted Review Keyword: Very good
Number of services: 6
Extracted Hotel Data: {'Location': 'Seoul, South Korea', 'Hotel Name': 'Ocloud Hotel Gangnam', 'Stars': 4, 'Rating': '8.4', 'Review Keyword': 'Very good', 'Services': ['Parking available', 'Gym', 'Laundry', 'Housekeeping', 'Air conditioning', 'Free WiFi'], 'Property Offers': [{'Room Name': 'Standard Double Room', 'Room Charges': '$85'}, {'Room Name': 'Deluxe Double Room', 'Room Charges': '$88'}, {'Room Name': 'Deluxe King Room', 'Room Charges': '$88'}, {'Room Name': 'Ocloud Suite Room', 'Room Charges': '$155'}]}

Processing link 13
Extracted Hotel Name: Conrad Seoul
Extracted Review Keyword: Wonderful
Number of services: 6
Error extracting room charges for offer 15: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-stid="price-summary"]"}
  (Session info: chrome=131.0.6778.140); For documentation on this error, please v


Processing link 16
Extracted Hotel Name: Shilla Stay Yeoksam
Extracted Review Keyword: Excellent
Number of services: 6
Error extracting room charges for offer 12: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-stid="price-summary"]"}
  (Session info: chrome=131.0.6778.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00BA34A3+25059]
	(No symbol) [0x00B2CEA4]
	(No symbol) [0x00A0BEC3]
	(No symbol) [0x00A4FD86]
	(No symbol) [0x00A4FFCB]
	(No symbol) [0x00A45B01]
	(No symbol) [0x00A71F44]
	(No symbol) [0x00A45A24]
	(No symbol) [0x00A72194]
	(No symbol) [0x00A8B51E]
	(No symbol) [0x00A71C96]
	(No symbol) [0x00A43FAC]
	(No symbol) [0x00A44F3D]
	GetHandleVerifier [0x00E95613+3113811]
	GetHandleVerifier [0x00EAA2DA+3199002]
	GetHandleVerifier [0x00EA2AB2+3168242]
	GetHandleVerifier [0x00C43310+680016]
	(No sy

...


Processing link 280
Extracted Hotel Name: Hotel Rian
Extracted Review Keyword: Very good
Number of services: 5
Extracted Hotel Data: {'Location': 'Seoul, South Korea', 'Hotel Name': 'Hotel Rian', 'Stars': 4, 'Rating': '8.2', 'Review Keyword': 'Very good', 'Services': ['24/7 front desk', 'Laundry', 'Air conditioning', 'Free WiFi', 'Business services'], 'Property Offers': [{'Room Name': 'Standard Double Room (Bigger than most in Seoul)', 'Room Charges': '$129'}, {'Room Name': 'Standard Twin Room (Bigger than most in Seoul)', 'Room Charges': '$136'}, {'Room Name': 'Deluxe Double Room', 'Room Charges': '$171'}, {'Room Name': 'Family Twin Room', 'Room Charges': '$183'}, {'Room Name': 'Deluxe Triple Room', 'Room Charges': '$234'}]}

Processing link 281
Extracted Hotel Name: Hotel Crescendo Seoul
Extracted Review Keyword: Good
Number of services: 6
Error extracting room charges for offer 6: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div[data-stid


Processing link 283
Extracted Hotel Name: Karak Tourist Hotel
Extracted Review Keyword: Good
Number of services: 6
Extracted Hotel Data: {'Location': 'Seoul, South Korea', 'Hotel Name': 'Karak Tourist Hotel', 'Stars': 3, 'Rating': '8.0', 'Review Keyword': 'Good', 'Services': ['Room service', 'Breakfast available', '24/7 front desk', 'Laundry', 'Housekeeping', 'Parking included'], 'Property Offers': [{'Room Name': 'Double Room', 'Room Charges': '$68'}, {'Room Name': 'Twin Room', 'Room Charges': '$75'}]}

Processing link 284
Extracted Hotel Name: Travelodge Myeongdong Euljiro
Extracted Review Keyword: Very good
Number of services: 6
Extracted Hotel Data: {'Location': 'Seoul, South Korea', 'Hotel Name': 'Travelodge Myeongdong Euljiro', 'Stars': 3, 'Rating': '8.4', 'Review Keyword': 'Very good', 'Services': ['Gym', '24/7 front desk', 'Breakfast available', 'Laundry', 'Air conditioning', 'Free WiFi'], 'Property Offers': [{'Room Name': 'Superior Twin Room', 'Room Charges': '$153'}, {'Room N


Processing link 290
Extracted Hotel Name: OYO Rooftop Hostel
Extracted Review Keyword: Very good
Number of services: 5
Extracted Hotel Data: {'Location': 'Seoul, South Korea', 'Hotel Name': 'OYO Rooftop Hostel', 'Stars': 3, 'Rating': '8.0', 'Review Keyword': 'Very good', 'Services': ['Breakfast included', 'Laundry', 'Housekeeping', 'Air conditioning', 'Free WiFi'], 'Property Offers': [{'Room Name': 'Standard Double Room', 'Room Charges': '$87'}, {'Room Name': 'Quadruple Room', 'Room Charges': '$154'}, {'Room Name': 'Family Room (8 persons)', 'Room Charges': '$202'}]}

Processing link 291
Extracted Hotel Name: JW Marriott Dongdaemun Square Seoul
Extracted Review Keyword: Wonderful
Number of services: 6
Extracted Hotel Data: {'Location': 'Seoul, South Korea', 'Hotel Name': 'JW Marriott Dongdaemun Square Seoul', 'Stars': 5, 'Rating': '9.0', 'Review Keyword': 'Wonderful', 'Services': ['Breakfast available', 'Hot Tub', 'Bar', 'Pool', 'Spa', 'Parking included'], 'Property Offers': [{'Room N


Processing link 295
Extracted Hotel Name: Fox Hotel
Error extracting review keyword: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.uitk-text.uitk-type-500.uitk-type-medium.uitk-text-default-theme"}
  (Session info: chrome=131.0.6778.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00BA34A3+25059]
	(No symbol) [0x00B2CEA4]
	(No symbol) [0x00A0BEC3]
	(No symbol) [0x00A4FD86]
	(No symbol) [0x00A4FFCB]
	(No symbol) [0x00A45B01]
	(No symbol) [0x00A71F44]
	(No symbol) [0x00A45A24]
	(No symbol) [0x00A72194]
	(No symbol) [0x00A8B51E]
	(No symbol) [0x00A71C96]
	(No symbol) [0x00A43FAC]
	(No symbol) [0x00A44F3D]
	GetHandleVerifier [0x00E95613+3113811]
	GetHandleVerifier [0x00EAA2DA+3199002]
	GetHandleVerifier [0x00EA2AB2+3168242]
	GetHandleVerifier [0x00C43310+680016]
	(No symbol) [0x00B357ED]
	(No symbol) [0x00B32


Processing link 300
Extracted Hotel Name: The First Stay Hotel
Extracted Review Keyword: Very good
Number of services: 6
Extracted Hotel Data: {'Location': 'Seoul, South Korea', 'Hotel Name': 'The First Stay Hotel', 'Stars': 3, 'Rating': '8.4', 'Review Keyword': 'Very good', 'Services': ['Housekeeping', 'Parking available', 'Air conditioning', 'Laundry', '24/7 front desk', 'Free WiFi'], 'Property Offers': [{'Room Name': 'Standard Twin Room', 'Room Charges': '$112'}, {'Room Name': 'Standard Double Room', 'Room Charges': '$115'}, {'Room Name': 'Deluxe Double Room', 'Room Charges': '$118'}, {'Room Name': 'Deluxe Twin Room', 'Room Charges': '$115'}, {'Room Name': 'Family Room', 'Room Charges': '$131'}, {'Room Name': 'Suite, Corner', 'Room Charges': '$143'}, {'Room Name': 'Executive Suite', 'Room Charges': '$154'}]}
Data exported to hotel_data_Seoul_South_Korea.csv
